In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import os

batch_size = 64
num_classes = 10
epochs = 200
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_main, y_main), (x_test, y_test) = cifar10.load_data()

print('x shape:', x_main.shape)
x_train=x_main[:40000]
y_train=y_main[:40000]
print('x_train shape:', x_train.shape)
x_val=x_main[40000:]
y_val=y_main[40000:]
print('x_val shape:', x_val.shape)
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

x shape: (50000, 32, 32, 3)
x_train shape: (40000, 32, 32, 3)
x_val shape: (10000, 32, 32, 3)


In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [4]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255

In [5]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model_2xLR.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [6]:

print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_val, y_val),
                    steps_per_epoch=5000,callbacks=[es,mc])

Using real-time data augmentation.
Epoch 1/200
5000/5000 [==============================] - 109s 22ms/step - loss: 1.5520 - acc: 0.4366 - val_loss: 1.2193 - val_acc: 0.5690

Epoch 00001: val_acc improved from -inf to 0.56900, saving model to best_model_2xLR.h5
Epoch 2/200
5000/5000 [==============================] - 163s 33ms/step - loss: 1.2060 - acc: 0.5719 - val_loss: 0.9936 - val_acc: 0.6490

Epoch 00002: val_acc improved from 0.56900 to 0.64900, saving model to best_model_2xLR.h5
Epoch 3/200
5000/5000 [==============================] - 194s 39ms/step - loss: 1.0489 - acc: 0.6309 - val_loss: 0.8579 - val_acc: 0.6996

Epoch 00003: val_acc improved from 0.64900 to 0.69960, saving model to best_model_2xLR.h5
Epoch 4/200
5000/5000 [==============================] - 195s 39ms/step - loss: 0.9510 - acc: 0.6661 - val_loss: 0.7970 - val_acc: 0.7227

Epoch 00004: val_acc improved from 0.69960 to 0.72270, saving model to best_model_2xLR.h5
Epoch 5/200
5000/5000 [=============================

5000/5000 [==============================] - 102s 20ms/step - loss: 0.6997 - acc: 0.7710 - val_loss: 0.6063 - val_acc: 0.8013

Epoch 00041: val_acc did not improve from 0.81240
Epoch 42/200
5000/5000 [==============================] - 102s 20ms/step - loss: 0.7001 - acc: 0.7695 - val_loss: 0.7344 - val_acc: 0.7574

Epoch 00042: val_acc did not improve from 0.81240
Epoch 43/200
5000/5000 [==============================] - 103s 21ms/step - loss: 0.7072 - acc: 0.7695 - val_loss: 0.7249 - val_acc: 0.7568

Epoch 00043: val_acc did not improve from 0.81240
Epoch 44/200
5000/5000 [==============================] - 102s 20ms/step - loss: 0.7063 - acc: 0.7692 - val_loss: 0.5910 - val_acc: 0.8007

Epoch 00044: val_acc did not improve from 0.81240
Epoch 45/200
5000/5000 [==============================] - 103s 21ms/step - loss: 0.7136 - acc: 0.7666 - val_loss: 0.6305 - val_acc: 0.7960

Epoch 00045: val_acc did not improve from 0.81240
Epoch 46/200
5000/5000 [==============================] - 103s 

5000/5000 [==============================] - 171s 34ms/step - loss: 1.0987 - acc: 0.6504 - val_loss: 0.8857 - val_acc: 0.7025

Epoch 00084: val_acc did not improve from 0.81240
Epoch 85/200
5000/5000 [==============================] - 187s 37ms/step - loss: 1.1176 - acc: 0.6461 - val_loss: 1.0751 - val_acc: 0.6653

Epoch 00085: val_acc did not improve from 0.81240
Epoch 86/200
5000/5000 [==============================] - 144s 29ms/step - loss: 1.1210 - acc: 0.6436 - val_loss: 0.8650 - val_acc: 0.7130

Epoch 00086: val_acc did not improve from 0.81240
Epoch 87/200
5000/5000 [==============================] - 103s 21ms/step - loss: 1.1290 - acc: 0.6402 - val_loss: 0.9591 - val_acc: 0.6810

Epoch 00087: val_acc did not improve from 0.81240
Epoch 88/200
5000/5000 [==============================] - 106s 21ms/step - loss: 1.1347 - acc: 0.6385 - val_loss: 1.0049 - val_acc: 0.6535

Epoch 00088: val_acc did not improve from 0.81240
Epoch 89/200
5000/5000 [==============================] - 107s 

KeyboardInterrupt: 

In [7]:
x_test = x_test.astype('float32')
x_test /= 255
y_test = keras.utils.to_categorical(y_test, num_classes)
model.evaluate(x=x_test, y=y_test, batch_size=batch_size, verbose=1)

10000/10000 [==============================] - 0s 44us/step


[0.9591198930740357, 0.6793]

In [8]:
del model

In [9]:
from keras.models import load_model
model= load_model('best_model_2xLR.h5')

In [10]:
model.evaluate(x=x_test, y=y_test, batch_size=batch_size, verbose=1)

10000/10000 [==============================] - 0s 49us/step


[0.5958372214794159, 0.8054]